In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#!pip install iterative-stratification
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import os

#!pip install iterative-stratification



In [ ]:
os.listdir()

In [ ]:

path1='../input/lish-moa'
os.chdir(path1)


In [ ]:
os.listdir()

In [ ]:
os.listdir('/kaggle')

In [ ]:
os.listdir('/kaggle/input/pca-features/')

In [ ]:
features_pca=pd.read_csv('/kaggle/input/pca-features/pca_features.csv')
targets=pd.read_csv('/kaggle/input/targets/target_kfold.csv')



In [ ]:
os.listdir()

In [ ]:
feat=pd.read_csv('train_features.csv')
train_feat=pd.read_csv('train_features.csv')
test_prediction_feat=pd.read_csv('test_features.csv')
targets=pd.read_csv('train_targets_scored.csv')


*Data preprocess*

In [ ]:
gene_train = [col for col in train_feat.columns if 'g-' in col]
cell_train = [col for col in train_feat.columns if 'c-' in col]
train_g=train_feat[gene_train]
train_c=train_feat[cell_train]
train_category=train_feat[['cp_type', 'cp_time', 'cp_dose']]
tx=train_category['cp_type'].replace({'ctl_vehicle':0, 'trt_cp':1})
tx2=train_category['cp_dose'].replace({'D1':1, 'D2':2})
tx3=train_category['cp_time'].replace({24:1, 48:2, 72:3})
cp_time=pd.get_dummies(train_feat['cp_time'])
train_category2=pd.concat([tx,tx2, tx3], axis=1)
train_category22=pd.concat([tx,tx2, cp_time], axis=1)
test_g=test_prediction_feat[gene_train]
test_c=test_prediction_feat[cell_train]

test_cat=test_prediction_feat[['cp_type', 'cp_time', 'cp_dose']]
tx=test_cat['cp_type'].replace({'ctl_vehicle':0, 'trt_cp':1})
tx2=test_cat['cp_dose'].replace({'D1':1, 'D2':2})
tx3=test_cat['cp_time'].replace({24:1, 48:2, 72:3})
cp_time2=pd.get_dummies(test_cat['cp_time'])
test_category2=pd.concat([tx,tx2, tx3], axis=1)
test_category3 = test_category2[test_category2['cp_type'].notna()]
CP_T=test_category3['cp_type'].astype('int64')
CP_D=test_category3['cp_dose'].astype('int64')
c2=pd.concat([CP_T, CP_D, cp_time2], axis=1)

In [ ]:
test_tabnet=pd.concat([c2, test_g, test_c], axis=1)

In [ ]:
train_tabnet=pd.concat([train_category22,train_g, train_c], axis=1)

In [ ]:
train_tabnet.head()

Reduce features with PCA

In [ ]:
from sklearn.decomposition import PCA
#Keeping explaining 95% of variance
pca=PCA(0.95)
pca1=PCA(381)
pca2=PCA(44)

data_c=pd.concat([train_c, test_c], axis=0)
data_g=pd.concat([train_g, test_g], axis=0)
data_genes_pca=pca1.fit_transform(data_g)
data_cells_pca=pca2.fit_transform(data_c)


In [ ]:
data_genes_pca.shape

In [ ]:
data_cells_pca.shape

In [ ]:
id_x=train_feat[['sig_id']]
pc_c=['PC-cell-'+str(n) for n in range(data_cells_pca.shape[1])]
pc_g=['PC-gene-'+str(n) for n in range(data_genes_pca.shape[1])]

data_cells_pca=pd.DataFrame(data_cells_pca, columns=pc_c)
data_genes_pca=pd.DataFrame(data_genes_pca, columns=pc_g)

d_c_train=data_cells_pca.loc[:int(train_feat.shape[0])]
d_g_train=data_genes_pca.loc[:int(train_feat.shape[0])]

d_c_test=data_cells_pca.loc[int(train_feat.shape[0]):]
d_g_test=data_genes_pca.loc[int(train_feat.shape[0]):]

d_g_test.reset_index(drop=True, inplace=True)
d_c_test.reset_index(drop=True, inplace=True)

data_train_tabnet=pd.concat([id_x,train_category2, train_g, train_c], axis=1)
data_train_pca=pd.concat([id_x,train_category2, d_g_train, d_c_train], axis=1)
data_test=test_prediction_feat.drop(['sig_id', 'cp_type', 'cp_time', 'cp_dose'], axis=1)
data_test=pd.concat([id_x,test_category2, data_test], axis=1)
data_test_pca=pd.concat([id_x,test_category2,d_g_test, d_c_test], axis=1)



NN - Pytroch model

In [ ]:
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression


class MoaDataset:
    def __init__(self, dataset, features):
        self.dataset=dataset
        self.features=features
        

    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        return{
            'x':torch.tensor(self.dataset[item, :], dtype=torch.float),
            'y':torch.tensor(self.features[item, :], dtype=torch.float)
        }


class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

class Engine:
    def __init__(self, model, optimizer, device):

        self.model =model
        self.optimizer = optimizer
        self.device = device
    @staticmethod
    def loss_fn(targets, outputs):
        return nn.BCEWithLogitsLoss()(outputs, targets)

    def train(self, data_loader):
        self.model.train()
        final_loss=0
        train_accuracy=0

        for data in data_loader:
            self.optimizer.zero_grad()
            inputs= data['x'].to(self.device)
            targets= data['y'].to(self.device)
            outputs=self.model(inputs)
            #Calculate loss value
            loss=self.loss_fn(targets, outputs)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
            # y_pred=LR.predict(outputs.cpu().detach().numpy().round())
            # train_accuracy +=accuracy_score(targets.cpu().detach().numpy(), y_pred)
        return final_loss/len(data_loader)

    def validate(self, data_loader):
        self.model.eval()
        final_loss=0
        valid_accuracy=0

        for data in data_loader:
            inputs= data['x'].to(self.device)
            targets= data['y'].to(self.device)
            outputs=self.model(inputs)
            loss=self.loss_fn(targets, outputs)
            final_loss += loss.item()
            # valid_accuracy +=accuracy_score(targets.numpy(), outputs.numpy())

        return final_loss/len(data_loader) 

# class trainsform_hyperparameters():
#     def __init__(self, start_neurons, n_layers):
        
class Model(nn.Module):

    def __init__(self, num_features, num_targets, dropout1,dropout2,n_neuron1,n_neuron2):
        super().__init__()
        #Define model architecture
        self.model = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.utils.weight_norm(nn.Linear(num_features, n_neuron1)),
            nn.LeakyReLU(),

            nn.BatchNorm1d(n_neuron1),
            nn.Dropout(dropout1),
            nn.utils.weight_norm(nn.Linear(n_neuron1, n_neuron2)),
            nn.LeakyReLU(),


            nn.BatchNorm1d(n_neuron2),
            nn.Dropout(dropout2),
            nn.utils.weight_norm(nn.Linear(n_neuron2, num_targets)),


        )

    def forward(self, x):
        #Define how the data passes through the model 
        #in this case just straight through
        x = self.model(x)
        return x

    def load(self, model_path):
        self.load_state_dict(torch.load(model_path))
        self.eval()


In [ ]:
features_pca=pd.read_csv('/kaggle/input/pca-features/pca_features.csv')
targets=pd.read_csv('/kaggle/input/targets/target_kfold.csv')

In [ ]:
import torch
import numpy as np
import pandas as pd
import os
import time


DEVICE = "cuda"
EPOCHS = 340

folds=targets
feature_data=features_pca


#Takes the fold number as input 
def run_training(fold, lr_x,batch_size, dropout1,dropouit2, n_neur1, n_neur2):
    df = feature_data

    #folds = ann_data.train_y
    targets= folds.drop(['sig_id', 'kfold'], axis=1).columns
    features = df.drop('sig_id', axis=1).columns
    df=df.merge(folds, on='sig_id', how='left')

    #The following row states that the training data is 
    # all of the rows not contaning the current fold value
    train_df=df[df.kfold != fold].reset_index(drop=True)

    #Valid data rows wich are the same as fold
    valid_df=df[df.kfold == fold].reset_index(drop=True)

    x_train = train_df[features].to_numpy()
    x_valid = valid_df[features].to_numpy()
    y_train = train_df[targets].to_numpy()
    y_valid = valid_df[targets].to_numpy()
    
    train_dataset= MoaDataset(x_train, y_train)
    train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0)

    #Validate 
    valid_dataset= MoaDataset(x_valid, y_valid)
    valid_loader= torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=0)


    model = Model(428 , 206, dropout1,dropouit2, n_neur1, n_neur2)
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr_x)
    eng= Engine(model, optimizer, DEVICE)

    best_scores_kf=0
    #add early stopping and model saving here
    #Use underscore '_' as it is not used
    score=0
    score_v=0
    best_loss=1
    early_stopping= 20
    epochs_no_improve=0
    early_stop = False

    num_epochs_run=0
    for n in range(EPOCHS):
        num_epochs_run=n

        train_loss= eng.train(train_loader)
        valid_loss= eng.validate(valid_loader)
        score +=train_loss
        score_v +=valid_loss
        #Early stopping checking if model validation loss does imporve other wise stop after n steps.
        #Bstops if no improves is seen 
        if valid_loss < best_loss: 
            epochs_no_improve=0
            best_loss= valid_loss
            #print( f"Best loss: {best_loss} at epoch: {n}")
            torch.save(model.state_dict(),"/kaggle/working/best_weights.pth")
            if n==EPOCHS:
                print('1')
                
        else:
            epochs_no_improve += 1
            if epochs_no_improve == early_stopping:
                print(f'best epoch{valid_loss}')
                best_scores_kf=best_scores_kf+best_loss
                early_stop= True
                break
            else:
                continue
        if early_stop:
            print('Stopped')
            #torch.save(model.state_dict(),weights_path+'best_pytorch_weights.pth')
            break
    print(num_epochs_run)
    print(f"CV score valid: {best_loss}")
    return best_loss

def train_real(lr_x,batch_size, dropout1,dropout2, n_neur1, n_neur2):
    n_folds=5
    loss_tot=0
    for n in range(n_folds):
        loss_x=run_training(n,lr_x,batch_size,dropout1,dropout2, int(n_neur1), int(n_neur2))
        loss_tot +=loss_x
    print(f'Average CV-loss: {(loss_tot/n_folds)}')




def inference_fn(model, dataloader, device):
    model.eval()
    model.load_state_dict(torch.load("/kaggle/working/best_weights.pth"))
    pred= []
    model.to(DEVICE)
    for data in dataloader:
        inputs = data['x'].to(device)
        out= model(inputs)
        pred.append(out.sigmoid().detach().cpu().numpy()[0])
    return pred



def predict(x):
    DEVICE='cuda'
    x=x.drop(['sig_id'], axis=1)
    x=x.to_numpy()
    x= TestDataset(x)
    test_loader= torch.utils.data.DataLoader(x, batch_size=1, num_workers=0)
    model = Model(428, 206, 0.503620,0.192174, 1960, 1085)
    model.load_state_dict(torch.load("/kaggle/working/best_weights.pth"))
    pred1= inference_fn(model, test_loader, DEVICE)
    print(pred1)
    return pred1


In [ ]:
train_real(0.00006890, 789, 0.893620, 0.292174, 1960, 1085)

In [ ]:
data_test_pc1=data_test_pca[data_test_pca['cp_dose'].notna()]

In [ ]:
p1=predict(data_test_pc1)

In [ ]:
sig_id_test=test_prediction_feat[['sig_id']]
test_prediction_feat.head()


In [ ]:
names_mech=[col for col in targets.columns if col not in ['sig_id','kfold']]

In [ ]:
d1=pd.DataFrame(p1, columns=names_mech)

In [ ]:
d1.head()

In [ ]:
submission = pd.concat([sig_id_test, d1], axis=1)

In [ ]:
submission.head()

In [ ]:
os.remove('/kaggle/working/best_weights.pth')

In [ ]:
submission.to_csv('submission.csv', index=False)